最后，通过封装好的框架，建立一个基准模型

In [1]:
import os
os.chdir('d:/future/panel_informer')

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import lr_scheduler, Adam, AdamW


AttributeError: module 'numpy' has no attribute '__version__'

In [ ]:
feature = pd.read_csv('data/feature.csv', index_col='Unnamed: 0')
feature.dropna(inplace=True)
feature = feature.set_index('trade_date')
for i in feature.columns:
    feature[i] = (feature[i] - feature[i].mean())/ feature[i].std()

label = pd.read_csv('data/label.csv', index_col='Unnamed: 0')
label = label.set_index('trade_date')
data = pd.concat([feature, label], axis = 1)

In [ ]:
from utils.random_split import RandomSplit

data_set = RandomSplit(data, 'cuda:0')
data_set.set_output(feature.columns, dtype = torch.float32, unfold = True)
data_set.set_output(['ch_next_week', 'down_probability', 'middle_probability', 'up_probability'], dtype = torch.float32, unfold = False)
train_set, validation_set, test_set = data_set.get_split_data(500, 250, 250, 30)

In [ ]:
from utils.hybrid_loss import HybridLoss
loss_fn = HybridLoss(alpha = 0.5, delta = 50)

In [ ]:
from utils.prediction_recorder import PredictionRecorder
recorder = PredictionRecorder(is_logits = True)

In [ ]:
import matplotlib.pyplot as plt

ImportError: DLL load failed while importing _path: 找不到指定的模块。

In [ ]:
from utils.train_animator import TrainAnimator
animator = TrainAnimator(figsize=(12,6))

ImportError: DLL load failed while importing _path: 找不到指定的模块。

In [ ]:
from utils.hybrid_decoder import HybridDecoder

class Baseline_LSTM(nn.Module):
    """循环神经网络模型"""
    def __init__(self, input_size, hidden_size,num_layers, dropout, **kwargs):
        super(Baseline_LSTM, self).__init__(**kwargs)
        self.device = 'cuda:0'
        self.process = nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_size,
            num_layers = num_layers,
            dropout = dropout,
            batch_first = True,
            # nonlinearity='relu',
        )
        self.output = HybridDecoder(dim_state = hidden_size)

    def forward(self, x):
        random_drop = np.random.randint(0, 20)
        x = x[:,random_drop:,:]
        return self.output(self.process(x)[0][:,-1,:])

In [ ]:
model = Baseline_LSTM(input_size = 17,
                      hidden_size = 17 * 2,
                      num_layers = 2,
                      dropout = 0.5,
                      ).to('cuda:0')

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay = 1e-1)
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)


In [ ]:
from utils.model_train import ModelTrain
train = ModelTrain(model = model,
                   train_set = train_set,
                   validation_set=validation_set,
                   test_set=test_set,
                   loss_fn = loss_fn,
                   optimizer = optimizer,
                   scheduler=scheduler,
                   recorder=recorder,
                   graph=animator,
                   )
train.epoch_train(10,100,10)

d:\future\panel_informer\utils\train_animator.py:64: UserWarning: The figure layout has changed to tight
  self.fig.tight_layout()


KeyboardInterrupt: 